In [ ]:
import argparse
import os
import shutil
from src.analysis.project_parser import parse_project
from src.analysis.dependency_resolver import resolve_dependencies
from src.analysis.feature_mapper import map_features_to_components
from src.generation.project_generator import generate_project
from src.generation.feature_integration import integrate_features

from src.models.prompt_templates import get_prompt_template
from src.utils.logger import logger
from src.utils.file_operations import read_file, write_file
from src.vector_database.db_builder import build_vector_database
from src.vector_database.db_query import query_vector_database


In [ ]:
from src.models.llm_inference import query_llm
def main():
    # Prompt for the model
    prompt = "Who is man's best friend?"
    # Query the LLM with or without vector database
    print("Querying the LLM...")
    try:
        response = query_llm(prompt)
        print("Generated Response:")
        print(response)
    except Exception as e:
        print(f"Error during LLM query: {e}")

if __name__ == "__main__":
    main()



In [ ]:
import os
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ibm import WatsonxLLM
from chromadb import PersistentClient
import torch

# Load credentials from .env file
load_dotenv()
WATSONX_APIKEY = os.getenv("WATSONX_APIKEY")
PROJECT_ID = os.getenv("PROJECT_ID")

if not WATSONX_APIKEY or not PROJECT_ID:
    raise ValueError("API key or Project ID is missing. Please check your .env file.")

def get_lang_chain_model(model_type, max_tokens, min_tokens, decoding_method, temperature):
    """
    Initializes and returns a WatsonxLLM instance with the specified parameters.
    """
    return WatsonxLLM(
        model_id=model_type,
        url="https://eu-gb.ml.cloud.ibm.com",
        project_id=PROJECT_ID,
        params={
            "max_new_tokens": max_tokens,
            "min_new_tokens": min_tokens,
            "decoding_method": decoding_method,
            "temperature": temperature,
        },
    )

def answer_questions_from_dataframe(question, collection_name, persist_directory="./notebooks/chroma_db"):
    """
    Answers a question using a LangChain model and retrieves relevant documents from a Chroma collection.
    """
    # Specify model parameters
    model_type = "meta-llama/llama-3-1-70b-instruct"
    max_tokens = 300
    min_tokens = 100
    decoding_method = "greedy"
    temperature = 0.7

    # Initialize the WatsonxLLM model
    model = get_lang_chain_model(model_type, max_tokens, min_tokens, decoding_method, temperature)

    # Use the same embedding model as in create_embeddings_and_store for consistency
    model_name = "sentence-transformers/all-mpnet-base-v2"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model_kwargs = {'device': device}
    encode_kwargs = {'normalize_embeddings': True}

    # Initialize the embedding model
    hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

    # Initialize Chroma Persistent Client
    chroma_client = PersistentClient(path=persist_directory)

    # Ensure the collection exists
    if collection_name not in [col.name for col in chroma_client.list_collections()]:
        raise ValueError(f"Collection '{collection_name}' does not exist. Make sure it is created.")
    collection = chroma_client.get_collection(name=collection_name)

    # Use Chroma vectorstore with the given collection
    vector_store = Chroma(
        collection_name=collection_name,
        embedding_function=hf,
        client=chroma_client,
    )

    # Create a retriever from the vectorstore
    retriever = vector_store.as_retriever(search_kwargs={"k": 5})

    # Build the RetrievalQA chain
    chain = RetrievalQA.from_chain_type(
        llm=model,
        chain_type="stuff",
        retriever=retriever,
        # input_key="question"  # This line is not needed
    )

    # Run the chain with the question
    # response_text = chain.invoke({"question": question})  # Incorrect key
    response_text = chain.invoke({"query": question})  # Correct key

    print("--------------------------------- Generated response -----------------------------------")
    print(response_text)
    print("*********************************************************************************************")

    return response_text

# Example Usage
question = "How to integrate the specified features into the project components?"

# Ensure consistent settings
persist_directory = "./notebooks/chroma_db"
collection_name = "my_vector_collection"

response = answer_questions_from_dataframe(question, collection_name, persist_directory)
print(response)

In [ ]:
from src.models.llm_inference import query_llm
# Prompt for the model
prompt = "Who is man's best friend?"
# Query the LLM with or without vector database
print("Querying the LLM...")
response = query_llm(prompt) 
print(response)

In [ ]:
from src.vector_database.db_load import load_vector_db
from src.models.llm_inference import query_llm

# Define the collection name and persist directory for the vector database
COLLECTION_NAME = "my_vector_collection"
PERSIST_DIRECTORY = "./notebooks/chroma_db"

# Load the vector database
print("Loading vector database...")
try:
    vector_db = load_vector_db(collection_name=COLLECTION_NAME, persist_directory=PERSIST_DIRECTORY)
    print("Vector database loaded successfully.")
except Exception as e:
    print(f"Failed to load vector database: {e}")
    vector_db = None

# Prompt for the model

question = "How to integrate the specified features into the project components?"

# Query the LLM with or without vector database
print("Querying the LLM...")
response = query_llm(prompt, vector_db=vector_db) 

print(response)

In [ ]:
from src.vector_database.db_load import load_vector_db
from src.models.llm_inference import query_llm

prompt = "How to integrate the specified features into the project components?"
# Define the persistent directory and collection name
persist_directory = "./notebooks/chroma_db"
collection_name = "my_vector_collection"

# Load the vector database
vector_db = load_vector_db(collection_name, persist_directory)

# Query the vector database using the `query_llm` function
response = query_llm(prompt=prompt, vector_db=vector_db)

# Print the response
print("Generated Response:")
print(response)

In [ ]:
prompt="Add a user authentication feature to the project"

In [ ]:
logger.info("Starting the Factory Feature program")
# Define paths
old_project_path = "project_old"
new_project_path = "project_new"

In [ ]:
# Step 1: Parse the old project
logger.info("Parsing the project structure")
project_data = parse_project(old_project_path)
vector_db = build_vector_database(project_data)
logger.info("Vector database created for the project")

In [ ]:
# Step 2: Resolve dependencies
logger.info("Resolving project dependencies")
dependencies = resolve_dependencies(old_project_path)
logger.info(f"Dependencies found: {dependencies.keys()}")

In [ ]:
# Step 3: Map features to project components
logger.info("Mapping feature request to project components")
feature_template = get_prompt_template()
project_context = "\n".join([f"{key}: {value}" for key, value in dependencies.items()])
feature_request = feature_template.format(feature_request=prompt, project_context=project_context)
relevant_docs = query_vector_database(vector_db, feature_request)
logger.info(f"Relevant documents for the feature request: {[doc.metadata['source'] for doc in relevant_docs]}")

In [ ]:
# Step 4: Generate new project structure
logger.info("Generating the updated project structure")
if os.path.exists(new_project_path):
    shutil.rmtree(new_project_path)
shutil.copytree(old_project_path, new_project_path)

In [ ]:
# Step 5: Integrate new features
logger.info("Integrating new features into the project")
for doc in relevant_docs:
    integrate_features([doc.metadata["source"]], f"# Feature: {prompt}\n{doc.page_content}")

In [ ]:
# Step 6: Generate additional files or modify existing ones
feature_instructions = [
    {"file": os.path.relpath(doc.metadata["source"], old_project_path), "content": f"# Feature: {prompt}\n{doc.page_content}"}
    for doc in relevant_docs
]

In [ ]:
feature_instructions

In [ ]:
generate_project(old_project_path, new_project_path, feature_instructions)
logger.info("Feature integration completed")
print(f"Project updated with the feature: {prompt}")
print(f"Updated project saved in: {new_project_path}")